# Introduction
This project shows the creation of a AI chatbot with the following characteristics:
1. It reads PDFs to gather information for the AI. This can be extended to look for other file formats (think of company sharepoint)
2. It converts the texts in the document into chunks.
3. The chunks are embedded with a model. This project uses all-MiniLM-L6-v2 for the semantic search and sentence similarity.
4. The embeddings are stored in a vector database. This project stores onto Pinecone cloud vector database, but others can be used, such as Chroma, QDrant, etc, or locally with FAISS
5. Prompts are created to direct the model.
6. Prompts and embeddings are passed to an AI model for further processing. This project uses OpenAI ChatGTP, but other can be used, such as Google Gemini, Meta Llama, or locally with GGUF (ex: Llama-3.2-3B-Instruct-Q2_K.gguf for low resources)
7. AI models returns a response.
8. Flask is used to create backend rest APIs.
9. A chatbot is created with nodejs, vite, and react.
10. The chabot receives the user message and invokes Flash rest APIs.

# Setup
1. GitHub:
Create a github repository: gen-ai-medical-bot
Add .gitignore = Python
Select MIT license
2. From terminal, run command: git clone https://github.com/lucho715/gen-ai-medical-chat-bot.git
3. Conda (Optional/Recommended): Set up conda environment: ```conda create -n gen-ai-medical-chat-bot python=3.10 -y```. Activate: ```conda activate gen-ai-medical-chat-bot```
4. Create folder structure. From terminal, run ```python template.py```
5. Complete setup.py
6. Complete requirements.txt. From terminal, run ```pip install -r requirements.txt```
7. Create folder 'data'. Download PDF Gale Encyclopedia of Medicine Vol. 1 (A-B) or the complete version The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf

In [ ]:
!pip install -r requirements.txt

# Read documents 

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# Function to load PDF (takes 3min with my cpu)
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf_file("data")

# Split documents into chunks

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)

# Download embeddings

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings

In [22]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
# 384 dimension
# Free. Using OpenAI Embeddings incur costs

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"} # or "cuda" if you have a GPU
)

In [23]:
# Test embedding
query_result = embeddings.embed_query("Hello")
print("length", len(query_result))

length 384


# Pinecone cloud vector database
1. Create free account
2. Copy API Key to .env PINECONE_API_KEY

Index can be created in Pinecone, or programatically in python
Create index. Choose Custom. Vector Type=Dense. Dimension=384. Metric=Cosine.

In [ ]:
from pinecone import Pinecone as pc
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
pc = pc(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("lucho715")

In [16]:
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 40000}},
 'total_vector_count': 40000}


# Embed chunks and store in Pinecone

In [18]:
from langchain_pinecone import Pinecone

# Pass index_name which is not the same as index. This is not as in documentation (took 6min)
# Uncomment to store new information
#vector_store = Pinecone.from_documents(documents=text_chunks, index_name="lucho715", embedding=embeddings)

# Load Pinecone index

In [ ]:
# Becarefule this Pinecone is from langchain because langchain renamed PineconeVectorStore to Pinecone
from langchain_pinecone import Pinecone

docsearch = Pinecone.from_existing_index(index_name="lucho715", embedding=embeddings)

# Retrieve information from Pinecone

In [24]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [25]:
retrieved_docs = retriever.invoke("What is cancer?")

In [26]:
retrieved_docs

[Document(metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 730.0, 'page_label': '701', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf', 'total_pages': 4505.0}, page_content='Cancer\nDefinition\nCancer is not just one disease, but a large group of\nalmost 100 diseases. Its two main characteristics are\nuncontrolled growth of the cells in the human body\nand the ability of these cells to migrate from the\noriginal site and spread to distant sites. If the spread\nis not controlled, cancer can result indeath.\nDescription\nOne out of every four deaths in the United States\nis from cancer. It is second only to heart disease as a\ncause of death in the states. About 1.2 million'),
 Document(metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page':

# LLM response

In [28]:
from langchain_openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(model="gpt-4o-mini", temperature=0.3)

In [6]:
!pip show langchain

Name: langchain
Version: 1.0.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /opt/miniconda3/envs/get-ai-medical-chat-bot/lib/python3.10/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 


In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

In [30]:
prompt_template = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {input}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """
prompt = ChatPromptTemplate.from_template(prompt_template)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
# Streamming response
full_response = ""
for chunk in rag_chain.stream({"input": "What is anemia?"}):
    if "answer" in chunk:
        print(chunk["answer"], end="", flush=True)
        full_response += chunk["answer"]

 Anemia is a condition characterized by a deficiency in the iron-carrying capacity of the blood, leading to a reduced number of red blood cells or hemoglobin. The most common type is iron deficiency anemia, which results in small, pale red blood cells. This condition can be congenital or acquired and affects millions of people worldwide.

In [35]:
response = rag_chain.invoke({"input": "What is abasdfjsaidof?"})
print(response["answer"])

 I don't know.


8. Move all def to helper.py
9. Move prompt_template to promt.py
10. Move creation of index and store embeddings to store_index.py
11. Create chat.html in templates folder and style.css in static
12. Create app.py with flask to consume the defs and display in webpage

# HTML part (nodejs, react)
1. Install nodejs if not yet. MacOS: ```brew install node``` Verfiy version with ```node -v``` and ```npm -v```
2. Create a folder: ```chatbot-reactjs```
3. In terminal run: ```npm create vite@latest ./chatbot-reactjs -- --template react``` choose no to experimental rolldown and yes to npm and start now
4. Open browser mentioned. ex: http://localhost:5173/
5. Delete folder 'assets', and files 'App.css', empty index.css
6. Optional install VScode extension: ES7+ React/Redux/React-Native snippets

# Chatbot UI creation
7. Start creating UI in App.jsx
8. Create folder 'components'. Add file ChatbotIcon.jsx, put and SVG icond.
9. Create the UI in HTML and CSS with App.jsx and index.css
10. Create file in 'components' ChatForm.jsx, and move the forms.
11. Create file in 'components' ChatMessage.jsx, and move the user-message.
12. Manage bot-user interactions and message history
13. Manage generate response from a model via REST API
14. Optional enhacements: scroll to bottom, open/close chat, manage errors

# Connect React to Python
1. From React, use fetch to connect to the python flask rest api
2. Python flask rest api receives the message
3. Python uses an AI model to process the message 


# Build chatbot UI
1. Run in terminal: ```npm build dev```
2. Vite will create a folder named ```dist``` containing the ```index.html```

# Connect Flask to Vite dist folder
1. Tell flash to use a folder ```add = Flask (__name__, static_folder='project-folder/dist', static_url_path='')```
2. Route root path to index file ```@app.route("/") def index(): return send_from_directory(app.static_folder, 'index.html')```


Connect 